In [ ]:
import pandas as pd
df = pd.read_csv("법령용어_데이터.csv")
df.head()

,법령용어명,법령내용
0,080착신과금사업자,"{'LsTrmService': {'법령용어일련번호': '4350393', '법령용어..."
1,080착신과금서비스,"{'LsTrmService': {'법령용어일련번호': '4350383', '법령용어..."
2,10기가급인터넷서비스,"{'LsTrmService': {'법령용어일련번호': ['4082648', '408..."
3,110콜센터,"{'LsTrmService': {'법령용어일련번호': '630294', '법령용어명..."
4,112,"{'LsTrmService': {'법령용어일련번호': '5113733', '법령용어..."


In [ ]:
len(df)

1089

In [ ]:
df.loc[80,'법령내용']

"{'LsTrmService': {'법령용어일련번호': ['5077141', '2430005'], '법령용어명_한글': ['4교대근무', '4교대근무'], '법령용어명_한자': ['4교대근무', '4교대근무'], '법령용어코드': ['011402', '011402'], '법령용어코드명': ['법령정의사전', '법령정의사전'], '출처': ['기상청 청원경찰 복무 및 징계규정[기상청훈령 제1117호, 2024.5.1., 일부개정]', '환경부 전일근무관서 교대근무에 관한 규정[환경부훈령 제1144호, 2015.2.27., 제정]'], '법령용어정의': ['주간근무, 야간근무, 휴무 및 비번을 일정한 순서에 따라 실시하는 것을 말한다.', '교대근무자가 주간근무, 야간근무, 휴무 및 비번을 일정한 순서에 따라  4일 또는 8일을 주기로 하여 실시하는 것을 말한다.']}}"

In [ ]:
type(df.loc[100,'법령내용'])

str

In [ ]:
system_prompt = '''당신은 법령 상담을 위한 챗봇 데이터를 생성하는 전문가입니다.

당신의 임무는 "법령용어(법령정의사전)"에 수록된 용어들을 바탕으로, 사용자가 자연스럽게 질문할 수 있는 문장을 작성하는 것입니다.

# 작성 규칙:
- 말투는 자연스럽고 일상적인 질문 형태로 작성할 것
- 문장은 짧고 간결하게, 대략적인 수준으로 작성할 것
- 다음과 같은 표현을 활용할 것: "뭐예요?", "뭔가요?", "설명해 주세요."
- 용어만 나열하지 말고, 반드시 문장 형태로 만들 것
- 사람을 지칭하는 경우에는 "누구인가요?", "누구를 말하는 건가요?"처럼 질문할 것
- 반드시 질문 문장만 출력하고, 다른 설명이나 안내 문구는 출력하지 말 것

# 예시:
입력: 1인당 GNI
출력: 1인당 GNI가 뭔가요?

입력: 1종 금융투자업자
출력: 1종 금융투자업자는 누구를 말하는 건가요?

이제 입력된 용어마다 위의 지침에 따라 질문 문장을 만들어주세요.
'''

In [ ]:
import openai
client = openai.OpenAI(api_key="KEY")

In [ ]:
def get_question_from_lawname(user_prompt):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '\n답변:'}
      ],
    temperature=0
  )
  return response.choices[0].message.content

In [ ]:
user_prompt = df['법령용어명'].loc[80]
print(user_prompt)
question_result = get_question_from_lawname(user_prompt)
print(question_result)

4교대근무
4교대근무가 뭐예요?


In [ ]:
df = df[:1000]

In [ ]:
# 1000개만 사용
user_inputs = df['법령용어명'].to_list()

In [ ]:
from tqdm import tqdm
result_lst = []
for user_prompt in tqdm(user_inputs):
  result_lst.append(get_question_from_lawname(user_prompt))

100%|██████████| 1000/1000 [09:23<00:00,  1.77it/s]


In [ ]:
df['question'] = result_lst

In [ ]:
df.tail()

,법령용어명,법령내용,question
995,exemption,"{'LsTrmService': {'법령용어일련번호': '618451', '법령용어명...",exemption이 뭐예요?
996,exercise,"{'LsTrmService': {'법령용어일련번호': '618084', '법령용어명...",exercise가 뭐예요?
997,exertion,"{'LsTrmService': {'법령용어일련번호': '618085', '법령용어명...",exertion이 뭐예요?
998,exhibit,"{'LsTrmService': {'법령용어일련번호': '617730', '법령용어명...",exhibit가 뭐예요?
999,exhibition,"{'LsTrmService': {'법령용어일련번호': '618856', '법령용어명...",exhibition이 뭐예요?


In [ ]:
system_prompt2 = '''당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.

주어진 "법령 용어(법령정의사전)"와 "사전식 풀이"를 기반으로, 고객에게 친절하고 정확하게 말하는 답변을 작성하세요.

두 가지 답변 케이스가 존재하며 무조건 파이썬의 dictionary 형식으로 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다. 따라서 주의하십시오.
```python``` 등 쓰지 마십시오.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지사사항을 따라 적지마십시오. 해당 지시사항에 따라 적절한 value를 채워넣으십시오.
해당 사항이 없다면 빈 문자열 또는 빈 리스트로 적어야 합니다. 임의로 '없음' 등을 적어서는 안 됩니다.
해당 사항이 없다면 리스트를 작성해야 하는 경우, 적을 항목이 없다면 빈 리스트 []로 작성하십시오. 그 안에 어떠한 값도 넣지 마십시오.

요구 조건:
- 너무 딱딱하거나 형식적인 표현은 피하세요.
- 문장은 자연스럽고 격식있게 전달하세요.
- 어려운 내용은 2~3문장 추가하여 쉽게 설명해주세요.

만약 "사전식 풀이"에 "출처" 키 값이 없으면 아래와 같이 작성합니다.

답변:
{"법령용어정의": "여기에는 해당 "법령용어정의"의 밸류값에서 요구 조건에 맞춰서 작성하세요. 중복된 내용은 적지마세요.",
"출처유무": false}

만약 "사전식 풀이"에 "출처" 키 값이 있으면 아래와 같이 작성합니다.

답변:
{"법령용어정의": "여기에는 해당 "법령용어정의"의 밸류값에서 요구 조건에 맞춰서 작성하세요. 중복된 내용은 적지마세요.",
"출처유무": true,
"출처": "각 출처 문자열에서 **대괄호( [ ) 앞까지만 잘라서**, 법령명만 리스트로 추출하세요."}

예시 입력 및 출력:
- 입력: {'LsTrmService': {'법령용어일련번호': ['5077141', '2430005'], '법령용어명_한글': ['4교대근무', '4교대근무'], '법령용어명_한자': ['4교대근무', '4교대근무'], '법령용어코드': ['011402', '011402'], '법령용어코드명': ['법령정의사전', '법령정의사전'], '출처': ['기상청 청원경찰 복무 및 징계규정[기상청훈령 제1117호, 2024.5.1., 일부개정]', '환경부 전일근무관서 교대근무에 관한 규정[환경부훈령 제1144호, 2015.2.27., 제정]'], '법령용어정의': ['주간근무, 야간근무, 휴무 및 비번을 일정한 순서에 따라 실시하는 것을 말한다.', '교대근무자가 주간근무, 야간근무, 휴무 및 비번을 일정한 순서에 따라  4일 또는 8일을 주기로 하여 실시하는 것을 말한다.']}}
- 출력:
{"법령용어정의": "주간근무, 야간근무, 휴무 및 비번을 일정한 순서에 따라 실시하는 것을 말합니다."
"출처유무": true,
"출처": ["기상청 청원경찰 복무 및 징계규정", "환경부 전일근무관서 교대근무에 관한 규정"]}
'''

In [ ]:
def get_answer_from_lawname(user_prompt):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
          {"role": "system", "content": system_prompt2},
          {"role": "user", "content": user_prompt + '\n답변:'}
      ],
    temperature=0
  )
  return response.choices[0].message.content

In [ ]:
user_prompt = df['법령내용'].loc[15]
print(user_prompt)
answer_result = get_answer_from_lawname(user_prompt)
print(answer_result)

{'LsTrmService': {'법령용어일련번호': '5283411', '법령용어명_한글': '119구조대', '법령용어명_한자': '119구조대', '법령용어코드': '011402', '법령용어코드명': '법령정의사전', '출처': '119구조ㆍ구급에 관한 법률[법률 제20543호, 2024.12.3., 일부개정]', '법령용어정의': '탐색 및 구조활동에 필요한 장비를 갖추고 소방공무원으로 편성된 단위조직을 말한다.'}}
{"법령용어정의": "119구조대는 탐색 및 구조활동에 필요한 장비를 갖추고 소방공무원으로 편성된 단위조직을 의미합니다. 쉽게 말해, 긴급 상황에서 사람들을 구조하기 위해 특별히 훈련된 소방관들로 구성된 팀이라고 생각하시면 됩니다.",
"출처유무": true,
"출처": ["119구조ㆍ구급에 관한 법률"]}


In [ ]:
user_inputs = df['법령내용'].to_list()

In [ ]:
from tqdm import tqdm
result_lst2 = []
for user_prompt in tqdm(user_inputs):
  result_lst2.append(get_answer_from_lawname(user_prompt))

100%|██████████| 1000/1000 [38:31<00:00,  2.31s/it]


In [ ]:
df['answer'] = result_lst2

In [ ]:
df.head()

,법령용어명,법령내용,question,answer
0,080착신과금사업자,"{'LsTrmService': {'법령용어일련번호': '4350393', '법령용어...",080착신과금사업자가 뭐예요?,"{""법령용어정의"": ""착신과금서비스를 제공하는 기간통신사업자를 의미합니다. 착신과금..."
1,080착신과금서비스,"{'LsTrmService': {'법령용어일련번호': '4350383', '법령용어...",080착신과금서비스가 뭐예요?,"{""법령용어정의"": ""080착신과금서비스는 전기통신사업법에 따라 공통서비스 식별번호..."
2,10기가급인터넷서비스,"{'LsTrmService': {'법령용어일련번호': ['4082648', '408...",10기가급인터넷서비스가 뭐예요?,"{""법령용어정의"": ""10기가급 인터넷 서비스는 하향 전송속도가 2.5Gbps 이상..."
3,110콜센터,"{'LsTrmService': {'법령용어일련번호': '630294', '법령용어명...",110콜센터가 뭐예요?,"{""법령용어정의"": ""110콜센터는 정부에서 운영하는 콜센터를 의미합니다. 정부와 ..."
4,112,"{'LsTrmService': {'법령용어일련번호': '5113733', '법령용어...",112가 뭐예요?,"{""법령용어정의"": ""112는 「전기통신사업법」 제48조에 따른 전기통신번호자원 관..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   법령용어명     1000 non-null   object
 1   법령내용      1000 non-null   object
 2   question  1000 non-null   object
 3   answer    1000 non-null   object
dtypes: object(4)
memory usage: 31.4+ KB


In [ ]:
import json
# True/False 변환 및 JSON 파싱
def parse_json(row):
    try:
        return json.loads(row)
    except json.JSONDecodeError:
        return None  # 파싱 실패 시 None 반환

# JSON 파싱 적용

df['parsed_result'] = df['answer'].apply(parse_json)
# 파싱 실패한 행 제거

df_cleaned = df.dropna(subset=['parsed_result']).reset_index(drop=True)

In [ ]:
df_cleaned

,법령용어명,법령내용,question,answer,parsed_result
0,080착신과금사업자,"{'LsTrmService': {'법령용어일련번호': '4350393', '법령용어...",080착신과금사업자가 뭐예요?,"{""법령용어정의"": ""착신과금서비스를 제공하는 기간통신사업자를 의미합니다. 착신과금...",{'법령용어정의': '착신과금서비스를 제공하는 기간통신사업자를 의미합니다. 착신과금...
1,080착신과금서비스,"{'LsTrmService': {'법령용어일련번호': '4350383', '법령용어...",080착신과금서비스가 뭐예요?,"{""법령용어정의"": ""080착신과금서비스는 전기통신사업법에 따라 공통서비스 식별번호...",{'법령용어정의': '080착신과금서비스는 전기통신사업법에 따라 공통서비스 식별번호...
2,10기가급인터넷서비스,"{'LsTrmService': {'법령용어일련번호': ['4082648', '408...",10기가급인터넷서비스가 뭐예요?,"{""법령용어정의"": ""10기가급 인터넷 서비스는 하향 전송속도가 2.5Gbps 이상...",{'법령용어정의': '10기가급 인터넷 서비스는 하향 전송속도가 2.5Gbps 이상...
3,110콜센터,"{'LsTrmService': {'법령용어일련번호': '630294', '법령용어명...",110콜센터가 뭐예요?,"{""법령용어정의"": ""110콜센터는 정부에서 운영하는 콜센터를 의미합니다. 정부와 ...",{'법령용어정의': '110콜센터는 정부에서 운영하는 콜센터를 의미합니다. 정부와 ...
4,112,"{'LsTrmService': {'법령용어일련번호': '5113733', '법령용어...",112가 뭐예요?,"{""법령용어정의"": ""112는 「전기통신사업법」 제48조에 따른 전기통신번호자원 관...",{'법령용어정의': '112는 「전기통신사업법」 제48조에 따른 전기통신번호자원 관...
...,...,...,...,...,...
995,exemption,"{'LsTrmService': {'법령용어일련번호': '618451', '법령용어명...",exemption이 뭐예요?,"{""법령용어정의"": ""면제라는 것은 특정한 의무나 책임을 면하게 해주는 것을 의미합...",{'법령용어정의': '면제라는 것은 특정한 의무나 책임을 면하게 해주는 것을 의미합...
996,exercise,"{'LsTrmService': {'법령용어일련번호': '618084', '법령용어명...",exercise가 뭐예요?,"{""법령용어정의"": ""여기서 'exercise'는 '행사'라는 의미로 사용됩니다. ...",{'법령용어정의': '여기서 'exercise'는 '행사'라는 의미로 사용됩니다. ...
997,exertion,"{'LsTrmService': {'법령용어일련번호': '618085', '법령용어명...",exertion이 뭐예요?,"{""법령용어정의"": ""exertion은 행사(行使)를 의미합니다. 이는 어떤 권리나...",{'법령용어정의': 'exertion은 행사(行使)를 의미합니다. 이는 어떤 권리나...
998,exhibit,"{'LsTrmService': {'법령용어일련번호': '617730', '법령용어명...",exhibit가 뭐예요?,"{""법령용어정의"": ""exhibit은 법적 맥락에서 '증거물'을 의미합니다. 이는 ...",{'법령용어정의': 'exhibit은 법적 맥락에서 '증거물'을 의미합니다. 이는 ...


In [ ]:
system_prompt = '''당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.

당신의 역할은 아래 두 가지 작업을 수행하는 것입니다:

1. 주어진 "법령 용어(법령정의사전)"를 기반으로, 고객이 자연스럽게 질문할 법한 문장을 생성하세요.
2. 주어진 "법령 용어"의 "사전식 풀이"를 바탕으로, 고객에게 친절하고 정확하게 설명하는 답변을 작성하세요.

---

[1] 질문 문장 생성 규칙:
- 말투는 자연스럽고 일상적인 질문 형태로 작성할 것
- 문장은 짧고 간결하게, 대략적인 수준으로 작성할 것
- 다음과 같은 표현을 활용할 것: "뭐예요?", "뭔가요?", "설명해 주세요."
- 용어만 나열하지 말고, 반드시 문장 형태로 만들 것
- 사람을 지칭하는 경우에는 "누구인가요?", "누구를 말하는 건가요?"처럼 질문할 것
- 반드시 질문 문장만 출력하고, 다른 설명이나 안내 문구는 출력하지 말 것

예시:
- 입력: 1인당 GNI → 출력: 1인당 GNI가 뭔가요?
- 입력: 1종 금융투자업자 → 출력: 1종 금융투자업자는 누구를 말하는 건가요?

---

[2] 정의 설명 및 출처 처리 규칙:
- 두 번째 작업의 결과는 반드시 파이썬의 dictionary 형식으로 출력해야 합니다.
- 큰따옴표(") 안에 작은따옴표(')나 다른 따옴표를 넣지 마세요. JSON 파싱 오류가 발생합니다.
- 해당 항목이 없을 경우에는 "없음"이라는 표현 대신, 빈 문자열("") 또는 빈 리스트([])를 사용해야 합니다.

요구 조건:
- 딱딱하거나 기계적인 문장은 피하고, 자연스럽고 격식 있는 설명을 작성하세요.
- 어려운 정의는 2~3문장으로 풀어 설명해 주세요. 단, 중복된 표현은 삼가세요.
- 정의 설명 결과는 "법령용어정의"라는 키에 담아야 합니다.

출처 관련 규칙:
- 출처가 없을 경우:
  {"법령용어정의": "...", "출처유무": false}

- 출처가 있을 경우:
  {"법령용어정의": "...", "출처유무": true, "출처": [각 출처 문자열에서 대괄호([) 앞까지 잘라낸 법령명 리스트]}

예시 입력 및 출력:
- 입력: {'LsTrmService': {'법령용어일련번호': ['5077141', '2430005'], '법령용어명_한글': ['4교대근무', '4교대근무'], '법령용어명_한자': ['4교대근무', '4교대근무'], '법령용어코드': ['011402', '011402'], '법령용어코드명': ['법령정의사전', '법령정의사전'], '출처': ['기상청 청원경찰 복무 및 징계규정[기상청훈령 제1117호, 2024.5.1., 일부개정]', '환경부 전일근무관서 교대근무에 관한 규정[환경부훈령 제1144호, 2015.2.27., 제정]'], '법령용어정의': ['주간근무, 야간근무, 휴무 및 비번을 일정한 순서에 따라 실시하는 것을 말한다.', '교대근무자가 주간근무, 야간근무, 휴무 및 비번을 일정한 순서에 따라  4일 또는 8일을 주기로 하여 실시하는 것을 말한다.']}}
- 출력:
{"법령용어정의": "주간근무, 야간근무, 휴무 및 비번을 일정한 순서에 따라 실시하는 것을 말합니다."
"출처유무": true,
"출처": ["기상청 청원경찰 복무 및 징계규정", "환경부 전일근무관서 교대근무에 관한 규정"]}

'''


In [ ]:
df_cleaned['system_prompt'] = system_prompt
df_cleaned['user_prompt'] = df_cleaned['question']
df_cleaned['assistant'] = df_cleaned['parsed_result']

df_cleaned[['system_prompt', 'user_prompt', 'assistant']]

,system_prompt,user_prompt,assistant
0,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,080착신과금사업자가 뭐예요?,{'법령용어정의': '착신과금서비스를 제공하는 기간통신사업자를 의미합니다. 착신과금...
1,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,080착신과금서비스가 뭐예요?,{'법령용어정의': '080착신과금서비스는 전기통신사업법에 따라 공통서비스 식별번호...
2,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,10기가급인터넷서비스가 뭐예요?,{'법령용어정의': '10기가급 인터넷 서비스는 하향 전송속도가 2.5Gbps 이상...
3,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,110콜센터가 뭐예요?,{'법령용어정의': '110콜센터는 정부에서 운영하는 콜센터를 의미합니다. 정부와 ...
4,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,112가 뭐예요?,{'법령용어정의': '112는 「전기통신사업법」 제48조에 따른 전기통신번호자원 관...
...,...,...,...
995,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,exemption이 뭐예요?,{'법령용어정의': '면제라는 것은 특정한 의무나 책임을 면하게 해주는 것을 의미합...
996,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,exercise가 뭐예요?,{'법령용어정의': '여기서 'exercise'는 '행사'라는 의미로 사용됩니다. ...
997,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,exertion이 뭐예요?,{'법령용어정의': 'exertion은 행사(行使)를 의미합니다. 이는 어떤 권리나...
998,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,exhibit가 뭐예요?,{'법령용어정의': 'exhibit은 법적 맥락에서 '증거물'을 의미합니다. 이는 ...


In [ ]:
output_file = "train_data.json"

df_cleaned[['system_prompt', 'user_prompt', 'assistant']].to_json(
    output_file, orient="records", force_ascii=False, indent=4
) #orient="records" : 각 행을 하나의 JSON 객체로 표현

In [ ]:
from huggingface_hub import login

In [ ]:
# API 토큰으로 로그인 (발급받은 토큰을 입력)
login("KEY")

In [ ]:
from datasets import Dataset, DatasetDict
from huggingface_hub import HfApi

# JSON 파일 경로
file_path = "train_data.json"

# JSON 파일 로드
dataset = Dataset.from_json(file_path)

# 데이터셋 미리보기 (선택 사항)
print(dataset)

# 데이터셋을 허깅페이스 허브에 업로드.
# 위에서 Key 값을 입력해서 login() 상태여야 데이터셋을 허깅페이스에 올릴 수 있습니다.
# 저는 허깅페이스 프로필 버튼 기준으로 'iamjoon'이라는 닉네임을 사용하고 있으므로 이렇게 올렸습니다.
# 여러분들은 '여러분들의 닉네임/파일명'으로 올리셔야 합니다.
dataset.push_to_hub("julietz/law_term_2")

print("Dataset uploaded successfully!")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['system_prompt', 'user_prompt', 'assistant'],
    num_rows: 1000
})


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset uploaded successfully!


In [ ]:
from datasets import load_dataset
import pandas as pd

# 데이터셋 다운로드
dataset = load_dataset("julietz/law_term_2")

# 판다스 데이터프레임으로 변환 (선택사항)
if 'train' in dataset:
    train_df = dataset['train'].to_pandas()

README.md:   0%|          | 0.00/486 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/168k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train_df

,system_prompt,user_prompt,assistant
0,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,080착신과금사업자가 뭐예요?,{'법령용어정의': '착신과금서비스를 제공하는 기간통신사업자를 의미합니다. 착신과금...
1,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,080착신과금서비스가 뭐예요?,{'법령용어정의': '080착신과금서비스는 전기통신사업법에 따라 공통서비스 식별번호...
2,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,10기가급인터넷서비스가 뭐예요?,{'법령용어정의': '10기가급 인터넷 서비스는 하향 전송속도가 2.5Gbps 이상...
3,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,110콜센터가 뭐예요?,{'법령용어정의': '110콜센터는 정부에서 운영하는 콜센터를 의미합니다. 정부와 ...
4,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,112가 뭐예요?,{'법령용어정의': '112는 「전기통신사업법」 제48조에 따른 전기통신번호자원 관...
...,...,...,...
995,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,exemption이 뭐예요?,{'법령용어정의': '면제라는 것은 특정한 의무나 책임을 면하게 해주는 것을 의미합...
996,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,exercise가 뭐예요?,{'법령용어정의': '여기서 'exercise'는 '행사'라는 의미로 사용됩니다. ...
997,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,exertion이 뭐예요?,{'법령용어정의': 'exertion은 행사(行使)를 의미합니다. 이는 어떤 권리나...
998,당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 ...,exhibit가 뭐예요?,{'법령용어정의': 'exhibit은 법적 맥락에서 '증거물'을 의미합니다. 이는 ...
